<a href="https://colab.research.google.com/github/ismachy/Introduction-to-ML/blob/main/HW_7_Problem_2_b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**B**

**1. Weight Decay:**

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
import time

In [ ]:
# Load and preprocess CIFAR-10 data
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
train_images, test_images = train_images / 255.0, test_images / 255.0
train_labels = to_categorical(train_labels, num_classes=10)
test_labels = to_categorical(test_labels, num_classes=10)

170498071/170498071 [==============================] - 2s 0us/step


In [ ]:
def identity_block(x, filters, weight_decay=0.001):
    f1, f2, f3 = filters

    x_shortcut = x

    # First component of main path
    x = layers.Conv2D(f1, (1, 1), padding='valid', kernel_regularizer=regularizers.l2(weight_decay))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    # Second component of main path
    x = layers.Conv2D(f2, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    # Third component of main path
    x = layers.Conv2D(f3, (1, 1), padding='valid', kernel_regularizer=regularizers.l2(weight_decay))(x)
    x = layers.BatchNormalization()(x)

    # Matching the dimensions of the shortcut
    x_shortcut = layers.Conv2D(f3, (1, 1), padding='valid', kernel_regularizer=regularizers.l2(weight_decay))(x_shortcut)
    x_shortcut = layers.BatchNormalization()(x_shortcut)

    # Adding the shortcut to the main path
    x = layers.add([x, x_shortcut])
    x = layers.Activation('relu')(x)

    return x


In [ ]:
input_shape = (32, 32, 3)
inputs = tf.keras.Input(shape=input_shape)

x = layers.Conv2D(16, (3, 3), padding='same')(inputs)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)

for _ in range(3):  # Number of blocks in ResNet-10
    x = identity_block(x, filters=[16, 16, 64], weight_decay=0.001)

for _ in range(3):
    x = identity_block(x, filters=[32, 32, 128], weight_decay=0.001)

for _ in range(3):
    x = identity_block(x, filters=[64, 64, 256], weight_decay=0.001)

x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(10, activation='softmax')(x)

In [ ]:
model = models.Model(inputs, x)

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the model
start_time = time.time()
history_weight_decay = model.fit(train_images, train_labels, epochs=300, validation_data=(test_images, test_labels))
end_time = time.time()

In [ ]:
# Evaluate the model
test_loss_weight_decay, test_acc_weight_decay = model.evaluate(test_images, test_labels)

In [ ]:
# Report results for Weight Decay
print("Results for Weight Decay:")
print(f"Training Time: {end_time - start_time} seconds")
print(f"Training Loss after 300 epochs: {history_weight_decay.history['loss'][-1]}")
print(f"Evaluation Accuracy after 300 epochs: {test_acc_weight_decay}")

Results for Weight Decay:
Training Time: 28496.107424736023 seconds
Training Loss after 300 epochs: 0.09998306140875
Evaluation Accuracy after 300 epochs: 0.6872000288963318


**2. Dropout:**

In [ ]:
# Build the ResNet-10 model with Dropout
def identity_block(x, filters, dropout_rate):
    f1, f2, f3 = filters

    x_shortcut = x

    x = layers.Conv2D(f1, (1, 1), padding='valid')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(f2, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(f3, (1, 1), padding='valid')(x)
    x = layers.BatchNormalization()(x)

    # Adjust the shortcut connection
    if x.shape[-1] != x_shortcut.shape[-1]:
        x_shortcut = layers.Conv2D(f3, (1, 1), padding='valid')(x_shortcut)
        x_shortcut = layers.BatchNormalization()(x_shortcut)

    x = layers.add([x, x_shortcut])
    x = layers.Activation('relu')(x)

    x = layers.Dropout(dropout_rate)(x)

    return x

# Build the ResNet-10 model
input_shape = (32, 32, 3)
inputs = tf.keras.Input(shape=input_shape)

x = layers.Conv2D(16, (3, 3), padding='same')(inputs)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)

for _ in range(3):  # Number of blocks in ResNet-10
    x = identity_block(x, filters=[16, 16, 64], dropout_rate=0.3)

for _ in range(3):
    x = identity_block(x, filters=[32, 32, 128], dropout_rate=0.3)

for _ in range(3):
    x = identity_block(x, filters=[64, 64, 256], dropout_rate=0.3)

x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(10, activation='softmax')(x)




In [ ]:
model = models.Model(inputs, x)

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
start_time = time.time()
history_dropout = model.fit(train_images, train_labels, epochs=300, validation_data=(test_images, test_labels))
end_time = time.time()

In [ ]:
# Evaluate the model
test_loss_dropout, test_acc_dropout = model.evaluate(test_images, test_labels)

In [ ]:
# Report results for Dropout
print("Results for Dropout:")
print(f"Training Time: {end_time - start_time} seconds")
print(f"Training Loss after 300 epochs: {history_dropout.history['loss'][-1]}")
print(f"Evaluation Accuracy after 300 epochs: {test_acc_weight_decay}")

Results for Dropout:
Training Time: 27650.169725179672 seconds
Training Loss after 300 epochs: 0.77836888877
Evaluation Accuracy after 300 epochs: 0.65785455878576


**3. Batch Normalization:**

In [ ]:
def identity_block(x, filters):
    f1, f2, f3 = filters

    x_shortcut = x

    x = layers.Conv2D(f1, (1, 1), padding='valid')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(f2, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(f3, (1, 1), padding='valid')(x)
    x = layers.BatchNormalization()(x)

    # Adjust the shortcut connection
    x_shortcut = layers.Conv2D(f3, (1, 1), padding='valid')(x_shortcut)
    x_shortcut = layers.BatchNormalization()(x_shortcut)

    x = layers.add([x, x_shortcut])
    x = layers.Activation('relu')(x)

    # Batch Normalization after the addition
    x = layers.BatchNormalization()(x)

    return x


# Build the ResNet-10 model
input_shape = (32, 32, 3)
inputs = tf.keras.Input(shape=input_shape)

x = layers.Conv2D(16, (3, 3), padding='same')(inputs)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)

for _ in range(3):  # Number of blocks in ResNet-10
    x = identity_block(x, filters=[16, 16, 64])

for _ in range(3):
    x = identity_block(x, filters=[32, 32, 128])

for _ in range(3):
    x = identity_block(x, filters=[64, 64, 256])

x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(10, activation='softmax')(x)


In [ ]:
model = models.Model(inputs, x)

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
start_time = time.time()
history_batch_norm = model.fit(train_images, train_labels, epochs=300, validation_data=(test_images, test_labels))
end_time = time.time()

In [ ]:
# Evaluate the model
test_loss_batch_norm, test_acc_batch_norm = model.evaluate(test_images, test_labels)

In [ ]:
# Report results for Batch Normalization
print("Results for Batch Normalization:")
print(f"Training Time: {end_time - start_time} seconds")
print(f"Training Loss after 300 epochs: {history_dropout.history['loss'][-1]}")
print(f"Evaluation Accuracy after 300 epochs: {test_acc_weight_decay}")

Results for Batch Normalization:
Training Time: 30651.516417741776 seconds
Training Loss after 300 epochs: 0.66578987789
Evaluation Accuracy after 300 epochs: 0.69785455878576
